# Introduction

## 1. Research topic
The aim of our project is to find out whether the product created from two designated molecules is stable. Indeed, our vision is to ask a user to draw two molecules of his choice. Thanks to the "reaction_utils" template, a product is created. To find out whether it was stable and whether the reaction was spontaneous, we also chose to calculate free enthalpies.

Free enthalpy, or Gibbs free energy, ΔG is a thermodynamic function that determines whether a chemical reaction can occur spontaneously at constant temperature and pressure. It is calculated by the formula ΔG=ΔH-TΔS where H is enthalpy [J], T is the temperature [K] and S is the entropy [J]. A reaction is spontaneous if the variation in free enthalpy is negative. In our case,  If the enthalpy of the sum of the reactants is greater than that of the product, then the latter is stable. This concept is therefore essential for predicting chemical equilibrium and optimizing the conditions of chemical reactions, whether industrial or biological. Free enthalpy thus enables us to better understand the direction and feasibility of chemical transformations.


## 2. Setting :
To this purpose, several aspects were analyzed. First, the goal was to analyze the free energy of each molecule. Indeed, if the sum of the reactants enthalpy is greater than the product, the reaction is spontaneous. On the contrary, the product is not stable and the reaction will not occur.
Therefore, the enthalpy was calculated using the following steps : when the user draw its reactants, we created a code which converted the draw reactants into smiles. Then, a function was programmed to convert smiles into coordonates 3D and an other function converted the latter in energy. Another part also involved creating the product using the template "reaction_utils" and follow the same steps as mentioned to get its free energy. 
All this stages can be summaryze with this chart : 

Molecule ( Smiles “CCOC…”) → Convert into 3D (xyz) → Found the energy using XTB and Morfeus

Another point of this project was to create an interface. Inspired by lecture 12, streamlit and ketcher were programmed. So, when the user draws the two reagents, they can see their drawing as well as the smiles, the 3D visualization and the energy of his molecules. The product is also illustrated with the same features.

# Project, Problems and Solutions

## 1. Project
## 2. Problems
## 3. Solutions

# 3. Results and Discussion


